In [1]:
import os
import numpy as np
import mxnet as mx
from mxnet import nd, gluon, init, autograd
from mxnet.gluon.data.vision import datasets
from mxnet.gluon import nn
from matplotlib import pyplot as plt
print("Load Package!")

Load Package!


In [13]:
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()
print(ctx)

cpu(0)

In [15]:
train_raw_data = datasets.MNIST(train=True)
val_raw_data = datasets.MNIST(train=False)

train_data = {}
train_data['data'] = np.array([i[0].asnumpy() for i in train_raw_data])
train_data['label'] = np.array([i[1] for i in train_raw_data])
#train_data['label'] = np.array([np.eye(1, 10, k=i[1]).squeeze(axis=0) for i in train_raw_data])

print(train_data['data'].shape)
print(train_data['label'].shape)

val_data = {}
val_data['data'] = np.array([i[0].asnumpy() for i in val_raw_data])
val_data['label'] = np.array([i[1] for i in val_raw_data])
#val_data['label'] = np.array([np.eye(1, 10, k=i[1]).squeeze(axis=0) for i in val_raw_data])

print(val_data['data'].shape)
print(val_data['label'].shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


In [ ]:
# %%
net = nn.Sequential()
net.add(nn.Conv2D(16, 3, padding=(1,1), activation='relu'))
net.add(nn.MaxPool2D())
net.add(nn.Conv2D(64, 3, padding=(1,1), activation='relu'))
net.add(nn.MaxPool2D())
net.add(nn.GlobalAvgPool2D())
net.add(nn.Dense(1, activation='sigmoid'))

net.initialize(init=init.Xavier())

cross_entropy = gluon.loss.SoftmaxCELoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01})
#%%
print("Setting Done!")

In [ ]:
batch_size = 100
tot_iter = len(train_data['data']) // batch_size
print("Start Training!")
for epoch in range(10):
    train_loss, train_acc, valid_acc = 0., 0., 0.
    #tic = time.time()
    # forward + backward
    for iter in range(tot_iter):
        idx = np.random.choice(len(train_data['data']), batch_size, replace=False)
        with autograd.record():
            output = net(nd.array(train_data['data'][idx]))
            loss = cross_entropy(output, nd.array(train_data['label'][idx]))
        loss.backward()
        # update parameters
        trainer.step(batch_size)
        train_loss += loss.mean().asscalar()
        
    val_idx = np.random.choice(len(val_data['data']), 100, replace=False)
    output = nd.argmax(net(nd.array(val_data['data'][val_idx])), axis = 1).asnumpy()
    acc = np.mean(output == val_data['label'][val_idx])

    print("Epoch : %d, loss : %f, val_acc : %f"%(epoch+1, train_loss/batch_size, acc))